In [ ]:
#passos para fazer ao analisar os dados
#adicionar dependencias
#carregar os dados dos dataset
#analisar de dados
#normalizar dados das colunas
#verificar dados nullos
#verificar dados categoricos
#verificar representatividade dos dados do objetivo em relação ao restante dos dados
#(objetivo = 1, restante = 0)
#normalizar os dados
#verificar separabilidade linear
#separar os dados treino / teste estratificando
#definir o baseline otimista / pessimista
#gerar os dados estatisticos
#otimizacao verificar os falsos positivos e falsos negativos em funcao do limiar minimo
#otimizacao verificar o precisao e recall em funcao do limiar minimo
#otimizacao verificar curva ROC vs classificador aleatorio
#otimizacao lucro medio em relacao ao limiar minimo
#gerar o modelo e os dados estatisticos
#comparar com os modelos otimista e pessimistas

In [33]:
import pandas as pd
from google.colab import drive

In [136]:
# esta variavel mantem o cache do dataset que foi carregado do gdrive
# para evitar carregar toda vez os dados
# caso precise carregar novamente os dados só rode este bloco e o cache será limpo
cache = {
    'dataset':None,
    'original_data_before_transformation':{}
}

In [137]:
def loadAnaliseAndTransformData(cache, configuration):
  data = loadDataSetFromGDriveOrCache(cache, configuration)

  if(configuration['dataset']['normalizeColumns']['enabled']):
    print()
    print('=> Normalizing column names...')
    normalizeColumnNames(configuration, data)
    print('=> Normalizing column names... OK')

  if(configuration['data_transformation']['enabled']):
    print()
    print('=> Applying data transformations...')
    transformData(configuration, cache, data)
    print('=> Applying data transformations... OK')

  showDataInfo(configuration, data)

  return data

def loadDataSetFromGDriveOrCache(cache, configuration):
  if(cache['dataset'] is None):
    print('=> Mounting drive...')
    drive.mount('/content/drive')
    print('=> Mounting drive... OK')

    print()
    print('=> Loading dataset...')
    data = pd.read_csv('/content/drive/MyDrive/'+configuration['dataset']['path'], sep=configuration['dataset']['sep'])
    cache['dataset'] = data
    print('=> Loading dataset... OK')
  else:
    print()
    print('=> Using cached dataset... OK')
  return cache['dataset']

def showDataInfo(configuration, data):
  if(configuration['dataset']['show_log']['dataset_head']):
    print()
    print('=> Dataset head: ')
    print(data.head())

  if(configuration['dataset']['show_log']['dataset_info']):
    print()
    print('=> Dataset info: ')
    print(data.info())

  if(configuration['dataset']['show_log']['dataset_describe']):
    print()
    print('=> Dataset describe: ')
    print(data.describe())

  if(configuration['dataset']['show_log']['dataset_nulls']):
    print()
    print('=> Dataset nulls: ')
    print(data.isnull().sum())

  if(configuration['dataset']['show_log']['dataset_duplicated']):
    print()
    print('=> Dataset duplicated: ')
    print(data.duplicated().sum())

  if(configuration['dataset']['show_log']['dataset_unique_values']):
    print()
    print('=> Dataset unique values: ')
    for column in data.columns:
      print('Column: '+column, data[column].unique())

  if(configuration['dataset']['show_log']['dataset_value_counts']):
    print()
    print('=> Dataset value counts: ')
    for column in data.columns:
      print('Column: '+column, data[column].value_counts())
  pass

def transformData(configuration, cache, data):
  for transformation in configuration['data_transformation']['transformations']:
    columnName = transformation['column']
    #backup original data column for cenarios where you need to run transformation multiple times
    #and not want to loose original data
    cachedColumns = cache['original_data_before_transformation']
    if(cachedColumns.get(columnName) is None):
      cachedColumns[columnName] = data[columnName]
    data[columnName] = cachedColumns[columnName].apply(transformation['function'])
  pass

def normalizeColumnNames(configuration, data):
  normalize = configuration['dataset']['normalizeColumns']
  if(normalize['modificator'] == 'lower'):
    data.columns = data.columns.str.lower()
  elif(normalize['modificator'] == 'upper'):
    data.columns = data.columns.str.upper()
  for column in data.columns:
    for replacement in normalize['replacements']:
      data[column] = data[column].replace(replacement['charToReplace'], replacement['replaceWith'])
  pass



In [139]:
#configuracao inicial
configuration = {
    'dataset':{
        'path':'ALGORITMOS DE IA/DATASET/drug200.csv',# CAMINHO PARA O SEU DATASET NO GDRIVE
        'sep':',',# SEPARADOR DOS DADOS DO SEU DATASET, POR EX: , OU ; OU OUTRO
        'normalizeColumns':{# SECAO PARA CONFIGURACAO DA NORMALIZACAO DAS COLUNAS
            'enabled':True,
            'modificator': 'lower',# ALLOWED ONLY 'lower' or 'upper' value
            'replacements':[
                {
                    'charToReplace':' ',
                    'replaceWith':'_'
                },
                {
                    'charToReplace':'(',
                    'replaceWith':''
                },
                {
                    'charToReplace':')',
                    'replaceWith':''
                },
            ]
        },
        'show_log':{# SECAO PARA CONFIGURACAO DOS LOGS
            'dataset_head':True,
            'dataset_info':False,
            'dataset_describe':False,
            'dataset_nulls':False,
            'dataset_duplicated':False,
            'dataset_unique_values':False,
            'dataset_value_counts':False,
        },
    },
    'data_transformation':{ #SECAO PARA CONFIGURACAO DA TRANSFORMACAO DOS DADOS
        'enabled':True,
        'transformations':[
            {
                'column':'sex',
                'function':lambda r:1 if r == 'M' else 0
            },
        ]
    }
}

data = loadAnaliseAndTransformData(cache, configuration)


=> Using cached dataset... OK

=> Normalizing column names...
=> Normalizing column names... OK

=> Applying data transformations...
=> Applying data transformations... OK

=> Dataset head: 
   age  sex      bp cholesterol  na_to_k   drug
0   23    0    HIGH        HIGH   25.355  drugY
1   47    1     LOW        HIGH   13.093  drugC
2   47    1     LOW        HIGH   10.114  drugC
3   28    0  NORMAL        HIGH    7.798  drugX
4   61    0     LOW        HIGH   18.043  drugY
